In [4]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-123371")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RA4HP7HMT to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-123371
Azure region: southcentralus
Subscription id: de8aba62-c352-42be-b980-2faedf08ead8
Resource group: aml-quickstarts-123371


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "C": uniform(0.001,100),
    "max_iter": choice(1,10,100,1000,10000)
})

# Specify a Policy
policy = BanditPolicy(slack_amount=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="training", 
                entry_script="train.py", 
                compute_target=cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(ps,
                                    primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=50,
                                    estimator=est,
                                    policy=policy)

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)

from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [22]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:',parameter_values[1])
print('\n Max iterations:',parameter_values[3])

model = best_run.register_model(model_name = 'best_hyperdrive_model', model_path = 'outputs/model.joblib')

Best Run Id:  HD_6a8e5d11-2bca-486c-83ff-5aefda226f0e_4

 Accuracy: 0.9150279193979121

 C: 54.278853267640976

 Max iterations: 100


In [12]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=data_path)

In [43]:
import sys  
sys.path.insert(0, './training')
from train import clean_data
from azureml.core.datastore import Datastore
import pandas

# Use the clean_data function to clean your data.
x = clean_data(ds)
x.head()
#data_store = Datastore(ws)
#x_ds = TabularDatasetFactory.register_pandas_dataframe(pandas.DataFrame(x), data_store, 'bankmarketing')

AttributeError: 'tuple' object has no attribute 'head'

In [40]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="regression",
    primary_metric="spearman_correlation",
    training_data=x_ds,
    label_column_name="y",
    n_cross_validations=3)

In [41]:
# Submit your automl run

automl_run = exp.submit(automl_config)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


DataException: DataException:
	Message: Expected column(s) y (label_column_name) not found in X.
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Expected column(s) y (label_column_name) not found in X.",
        "target": "label_column_name",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "MissingColumnsInData"
            }
        },
        "reference_code": "06328553-815b-486d-98dd-98e4c8952a7c"
    }
}

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###